---
title: Sentinel-2 burn scar analysis
---

# Sentinel-2 burn scar analysis

> To Do:
> - [ ] Convert script to notebook
> - [ ] Integrate outputs/figures in document

In [1]:
import geopandas as gpd
import rasterio as rio
from pathlib import Path


import boto3
import pystac_client

from rasterio.mask import mask
import datetime


In [ ]:
# Define search parameters
aoi_path = Path("data/vector/proposed_study_sites.geojson")
start_date = "2025-01-01"
end_date = "2025-03-31"
search_range = 7
start_datetime = datetime.datetime.strptime(start_date, "%Y-%m-%d")
end_datetime = datetime.datetime.strptime(end_date, "%Y-%m-%d")

# Calculate date ranges using search_range (in days)
start_range_min = (start_datetime - datetime.timedelta(days=search_range)).strftime(
    "%Y-%m-%d"
)
start_range_max = (start_datetime + datetime.timedelta(days=search_range)).strftime(
    "%Y-%m-%d"
)
end_range_min = (end_datetime - datetime.timedelta(days=search_range)).strftime(
    "%Y-%m-%d"
)
end_range_max = (end_datetime + datetime.timedelta(days=search_range)).strftime(
    "%Y-%m-%d"
)

time_range = [start_range_min, end_range_max]
collections = ["ga_s2am_ard_3", "ga_s2bm_ard_3", "ga_s2cm_ard_3"]
cloud_cover_threshold = 10
stac_api_url = "https://explorer.dea.ga.gov.au/stac/"

In [ ]:
aoi = gpd.read_file(aoi_path)

if aoi.crs != "EPSG:4326":
    aoi = aoi.to_crs("EPSG:4326")

geom = aoi.geometry.values[0].__geo_interface__

# Create a STAC client and search for imagery
catalog = pystac_client.Client.open(stac_api_url)
search = catalog.search(
    collections=collections,
    datetime=f"{time_range[0]}/{time_range[1]}",
    intersects=geom,
    max_items=100,
)

# Get all items
items = list(search.get_all_items())
print(f"Found {len(items)} items before filtering")

# Filter by cloud cover
filtered_items = []
for item in items:
    # Try different cloud cover property names
    cloud_cover = None
    for prop_name in ["eo:cloud_cover", "cloud_cover", "cloudy_pixel_percentage"]:
        if prop_name in item.properties:
            cloud_cover = item.properties[prop_name]
            break

    # Add to filtered list if below threshold
    if cloud_cover is not None and cloud_cover <= cloud_cover_threshold:
        filtered_items.append(item)

print(f"Found {len(filtered_items)} items with cloud cover <= {cloud_cover_threshold}%")


In [ ]:
# TODO: download all assets for each item
# TODO: calculate NBR for each item
# TODO: calculate dNBR for period
# TODO: apply USGS thresholds for burn severity
# TODO: save results to GeoTIFF
# TODO: convert to vector format and calculate area of burn severity classes
# TODO: make folium map with burn severity classes and study area boundary